In [4]:
from data_common.notebook import *  
import climate_scorecard_data_2022.la_processing

In [5]:
source_file = Path("data", "raw", "individual_answers.csv")

data_package_folder = Path("data", "data_packages", "scorecard_data")

split_data_package_folder = Path("data", "data_packages", "scorecards_by_council_group")

labels = {
    "s1_gov": "Governance, development and funding",
    "s2_m&a": "Mitigation and adaptation",
    "s3_c&a": "Commitment and intergration",
    "s4_coms": "Community, engagement and communications",
    "s5_mset": "Measuring and setting emissions targets",
    "s6_cb": "Co-benefits",
    "s7_dsi": "Diversity and inclusion",
    "s8_est": "Education, skills and training",
    "s9_ee": "Ecological emergency",
    "total": "Overall score",
}

weights = {
    "s1_gov": 0.15,
    "s2_m&a": 0.15,
    "s3_c&a": 0.15,
    "s4_coms": 0.15,
    "s5_mset": 0.10,
    "s6_cb": 0.05,
    "s7_dsi": 0.10,
    "s8_est": 0.10,
    "s9_ee": 0.05,
}


In [11]:
def score_row(row: pd.Series) -> float:
    value = 0
    for section, weight in weights.items():
        value += row[section] * weight
    return value

def df_to_score_columns(df, index="section"):
    pt = df.pivot_table(["score", "max_score"], index=[index], aggfunc="sum")
    pt.loc["raw_total"] = pt.sum()
    pt["percentage"] = pt["score"] / pt["max_score"]
    pt = pt[["percentage"]].transpose().reset_index(drop=True).fillna(0)
    pt.columns = list(pt.columns)
    return pt.iloc[0]

df = pd.read_csv(Path("data", "raw", "individual_answers.csv"))

df = df.groupby("local-authority-code").apply(df_to_score_columns)
# rdf = df[df["total"] > 0]
df["weighted_total"] = df.apply(score_row, axis="columns")
#df = (df.round(2) * 100).astype(int)

d = (
    df.sort_values("weighted_total", ascending=False)
    .reset_index()
    .la.get_council_info(["local-authority-type-name", "official-name"])
)
new_order = [x for x in d.columns if x != "official-name"]
d = d[["official-name"] + new_order]

filters = pd.read_csv(Path("data", "raw", "all_filters.csv")).drop(
    columns=["local-authority-type-name"]
)

d = (
    d.merge(filters, on="local-authority-code")
    .fillna("")
    .drop(columns=["raw_total", "la_deprivation_score"])
)

unitaries = [
    "Welsh unitary authority",
    "Scottish unitary authority",
    "Unitary authority",
    "Metropolitan district",
    "London borough",
    "City corporation",
]

league_map = {x: "Single tier" for x in unitaries}
league_map["NI district"] = "Northern Ireland"
league_map["County"] = "County councils"
league_map["Non-metropolitan district"] = "District councils"
league_map["Combined authority"] = "Combined/strategic authorities"
league_map["Strategic Regional Authority"] = "Combined/strategic authorities"

d["group"] = d["local-authority-type-name"].apply(lambda x: league_map.get(x, x))


In [12]:
pd.options.display.float_format = "{:,.4f}".format
d[d["local-authority-code"] == "STS"]

,official-name,local-authority-code,s1_gov,s2_m&a,s3_c&a,s4_coms,s5_mset,s6_cb,s7_dsi,s8_est,s9_ee,weighted_total,local-authority-type-name,council-quintile,pop_bucket,region,ruc_cluster,political_control,group
272,Staffordshire County Council,STS,0.29,0.0,0.2,0.44,0.2,0.25,0.0,0.6,0.25,0.24,County,2.0,800k - 1m,West Midlands,,Conservative,County councils
